In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import traci
import numpy as np
from tensorflow.keras import Sequential, layers
from tensorflow.random import set_seed
from collections import deque
from tensorflow import keras
from tensorflow import reduce_sum, reduce_mean, one_hot, GradientTape
from tensorflow.keras.losses import MeanSquaredError
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
from dotenv import load_dotenv
load_dotenv()


sumo_bin = os.getenv("SUMO")
#sumo_gui_bin = os.getenv("SUMO-GUI")
sumo_gui_bin = '/Library/Frameworks/EclipseSUMO.framework/Versions/1.22.0/EclipseSUMO/bin/sumo-gui'
simulConfig = os.getenv("SIMUL-CONFIG2")


print(sumo_gui_bin)

/Library/Frameworks/EclipseSUMO.framework/Versions/1.22.0/EclipseSUMO/bin/sumo-gui


In [3]:
from state import get_state, queue
from reward import calculate_reward, calculate_rewardV3

In [4]:
set_seed(42)  # extra code – ensures reproducibility on the CPU

input_shape = [184]  # == env.observation_space.shape
n_outputs = 16  # == env.action_space.n
#[((Phase(duration=30.0, state='GGrGrrGGrGrr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grGgrrgrGgrr', minDur=20.0, maxDur=20.0), Phase(duration=30.0, state='GrrGGrGrrGGr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grrgrGgrrgrG', minDur=20.0, maxDur=20.0)), [0, 2, 4, 6])]
model_action = Sequential([
    layers.Dense(128,activation=tf.keras.layers.LeakyReLU(alpha=0.01), input_shape=input_shape),
    layers.Dense(128, activation=tf.keras.layers.LeakyReLU(alpha=0.01)),
    layers.Dense(32, activation=tf.keras.layers.LeakyReLU(alpha=0.01)),
    layers.Dense(n_outputs, activation= 'linear')])

/Users/pr/.pyenv/versions/3.10.6/envs/sumo_env/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/Users/pr/.pyenv/versions/3.10.6/envs/sumo_env/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
def epsilon_greedy_policy(state, epsilon=0):
    if np.random.rand() < epsilon:
        return np.random.randint(n_outputs)  # random action
    else:

        Q_values = model_action.predict(state[np.newaxis], verbose=0)[0]
        print(Q_values)
        return Q_values.argmax()  # optimal action according to the DQN

In [6]:
def sample_experiences(batch_size):
    indices = np.random.randint(len(replay_buffer), size=batch_size)
    batch = [replay_buffer[index] for index in indices]
    states, actions, rewards, next_states = [
        np.array([experience[field_index] for experience in batch])
        for field_index in range(4)
    ]
    return states, actions, rewards, next_states

In [7]:
batch_size = 32
discount_factor = 0.5
optimizer = keras.optimizers.Adam(learning_rate=0.05)
loss_fn = MeanSquaredError()

def training_step(batch_size):
    experiences = sample_experiences(batch_size)
    states, actions, rewards, next_states = experiences  # a changer
    next_Q_values = model_action.predict(next_states, verbose=0)
    max_next_Q_values = next_Q_values.max(axis=1)
    # runs = 1.0 - (dones | truncateds)  # episode is not done or truncated
    target_Q_values = rewards + discount_factor * max_next_Q_values
    target_Q_values = target_Q_values.reshape(-1, 1)
    mask = one_hot(actions, n_outputs)
    with GradientTape() as tape:
        all_Q_values = model_action(states)
        Q_values = reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
        loss = reduce_mean(loss_fn(target_Q_values, Q_values))

    grads = tape.gradient(loss, model_action.trainable_variables)
    optimizer.apply_gradients(zip(grads, model_action.trainable_variables))

In [ ]:
replay_buffer = deque(maxlen=10000)
epoch = 500
nb_step = 80000



# PERIODE DE STABILITE

#traffic_scale = [0.1] * 100
# AUGMENTATION DE TRAFFIC
#traffic_scale.extend(np.linspace(0.1, 0.3, 40, dtype=float))
# OR
traffic_scale = 0.2



for episode in range(epoch):
    #SEED & TRAFFIC SCALING
    seed_value = np.random.randint(1000)
    #seed_value = 41



    sumoCmd = [sumo_bin, "-c",
               simulConfig,
               "--start",
               "--seed", str(seed_value),
               '--no-warnings',
               '--scale', str(np.random.uniform(0.1, 0.4))]




    if traci.isLoaded():
        try:
            traci.close()
        except traci.TraCIException:
            pass



    traci.start(sumoCmd)



    lane_ids =  traci.lane.getIDList()
    # Filter out internal edges (starting with ":")
    edge_ids = [e for e in traci.edge.getIDList() if not e.startswith(":")]


    trafic_light_ids = traci.trafficlight.getIDList()


    state=np.array(get_state(lane_ids))

    action=-1


    for step in range(nb_step):
        epsilon = max(1 - episode / epoch, 0.01)

        if step%2000 == 0:

            # Capture current statistics
            avg_speed = np.mean([traci.edge.getLastStepMeanSpeed(e) for e in edge_ids])
            halted_vehicles = np.sum([traci.edge.getLastStepHaltingNumber(e) for e in edge_ids])

            traci.simulationStep()  # Advance the simulation

            # Capture next-step statistics after simulation step
            next_avg_speed = np.mean([traci.edge.getLastStepMeanSpeed(e) for e in edge_ids])
            next_halted_vehicles = np.sum([traci.edge.getLastStepHaltingNumber(e) for e in edge_ids])

            next_state = np.array(get_state(lane_ids))
            reward = calculate_rewardV3(state, next_state, avg_speed, next_avg_speed, halted_vehicles, next_halted_vehicles)


            replay_buffer.append((state, action, reward, next_state))

            #########################################################################
            state=next_state
            action = epsilon_greedy_policy(state, epsilon)
            feu_1 = action // 4
            feu_2 = action % 4


            traci.trafficlight.setPhase(trafic_light_ids[0],2*feu_1)
            traci.trafficlight.setPhase(trafic_light_ids[1],2*feu_2)

            if len(replay_buffer) >= batch_size*10:
                training_step(batch_size)

        traci.simulationStep()
    print(f'episode : {episode}')
    traci.close()


 Retrying in 1 seconds


Step #720.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 153 ACT 33 BUF 0)                   #180.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 37 ACT 15 BUF 0)                   0.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 78 ACT 23 BUF 0)                    (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 113 ACT 14 BUF 0)                  episode : 0
Step #800.40 (0ms ?*RT. ?UPS, TraCI: 1ms, vehicles TOT 171 ACT 26 BUF 0)                  
 Retrying in 1 seconds


Step #720.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 174 ACT 21 BUF 0)                  episode : 1
Step #800.40 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 191 ACT 21 BUF 0)                  
 Retrying in 1 seconds


Step #720.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 302 ACT 47 BUF 0)                  episode : 2
Step #800.40 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 334 ACT 37 BUF 0)                  
 Retrying in 1 seconds


Step #720.00 (0m[-0.6900737  -0.21971965  0.7501506  -0.02178656 -0.20268634  0.0531821   
 -0.21407126  0.32944945 -0.399992    0.6159443   0.3921771   0.70861965
 -0.9181818  -0.06207876 -0.6781056  -0.21464393]
episode : 3
Step #800.40 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 210 ACT 25 BUF 0)                  
 Retrying in 1 seconds


...

[ 852.5411   642.56506 1189.7825   525.86884  671.65845  795.39014
  912.9303   999.2841  1150.9093   880.2231  -269.7118  1120.3479
  918.0622   593.32294  633.9859   935.1889 ]
[ 3133.2527  2904.6414  4330.931   2000.7922  2416.9758  2986.03
  3639.5864  3639.3298  4094.5112  3551.877  -1383.6277  4053.4702
  3888.3777  2237.4978  2599.2131  3424.5576]
[ 6972.288   6581.3164  9507.752   4395.968   5402.8413  6493.1055
  8239.553   8211.369   9004.109   8012.832  -3855.3708  9013.784
  8799.512   5028.761   5696.2847  7336.814 ]
[ 3882.75    3467.0312  5196.4355  2402.5916  2920.0142  3606.8306
  4434.387   4604.044   5007.0923  4447.6943 -1776.7589  4963.0605
  4659.622   2811.0964  3065.6492  3961.2644]
[ 6497.4863  5646.697   8654.278   3982.7493  5077.874   6003.007
  7302.56    7709.2266  8526.697   7331.7827 -2781.4028  8312.193
  7722.332   4695.7344  5059.144   6563.975 ]
[2910.494   2183.707   3768.8027  1773.7854  2339.1804  2659.216
 3012.1155  3481.3247  3935.267   3152.31

[ 6218.4033  5034.6567  8423.962   4141.511   5374.5503  5812.0522
  6637.0474  8128.8906  8894.564   6983.2188 -2108.7578  8483.929
  6978.826   4990.1157  4348.2583  6335.758 ]


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x106f1b970>>
Traceback (most recent call last):
  File "/Users/pr/.pyenv/versions/3.10.6/envs/sumo_env/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
KeyboardInterrupt: 


Interrupt signal received, trying to exit gracefully.T 21 ACT 18 BUF 0)                   
[ 6220.853   5151.1606  8403.012   4149.8716  5442.5664  5869.828
  6637.24    8044.0913  8936.761   6999.7197 -1817.3494  8439.013
  7094.111   4994.9546  4454.589   6444.0674]
[ 5117.988   4343.2124  7052.5435  3512.2444  4558.01    4904.495
  5459.8486  6765.786   7060.2812  5617.6045 -1698.0316  6919.6006
  5875.9976  4111.625   3648.1643  5487.914 ]
[ 5012.467   3982.6396  6867.326   3382.6738  4490.6997  4763.2085
  5198.5767  6676.6143  6905.833   5348.798  -1476.5569  6773.9414
  5487.774   3994.2305  3435.945   5339.771 ]
[3856.745  3157.293  5284.644  2565.3818 3627.5615 3649.6392 3831.433
 4968.97   4963.3335 3895.4897 -692.2483 4947.279  4326.202  2962.9243
 2724.5413 4231.825 ]
[2709.167   2125.3362  3667.108   1784.4269  2081.9814  2580.4592
 2622.6768  3554.0864  3178.5745  2807.4124  -148.65918 3366.6523
 2742.9617  2071.1675  1826.768   2951.9277 ]
[ 3809.7515  3567.8442  5278.86

[ 854.94617 1158.4727  1244.0775   641.72363  372.0929   843.65485
 1076.3046  1126.048    663.25525 1134.4703  -528.0431  1025.737
 1210.0012   671.454    750.1523  1029.306  ]
[ 4253.933   4349.2256  6063.5     2899.4285  3104.2375  3757.6123
  4642.9277  5780.1772  4311.5684  4537.846  -1622.347   5340.8174
  5177.3574  3120.3577  3202.1672  4804.245 ]
[ 5867.831   6041.0005  8403.119   4002.5564  4502.832   5078.6016
  6441.546   7966.0527  6088.3335  6185.1997 -2456.7847  7424.9985
  7286.1123  4277.4673  4437.9375  6617.6504]
[ 6746.7476  7074.2026  9693.063   4624.264   5222.0396  5765.9077
  7483.036   9143.957   7047.4033  7133.466  -3013.6772  8564.25
  8527.0205  4907.332   5157.354   7628.5654]


In [ ]:
rewards = []
best_score = 0
reward = 0
total_reward = 0
list_values = []
weights = []
wait_times = []
# replay_buffer = deque(maxlen=2000)

#SEED é SCALE
seed_value = np.random.randint(1000)
#seed_value = 41
traffic_scale = 0.2


#LAUNCH
sumoCmd = [sumo_gui_bin, "-c", simulConfig, "--start", "--seed", str(seed_value), '--scale', str(traffic_scale), '--gui-settings-file', 'full_state_sumo_gui_settings.xml']

if traci.isLoaded():
    traci.close()
traci.start(sumoCmd)
lane_ids =  traci.lane.getIDList()
# print(lane_ids[0])
# for lane in lane_ids:
#     print(traci.lane.getLastStepVehicleNumber(lane))
# north_lane = traci.lane.getLastStepVehicleNumber("N_0")
# south_lane = traci.lane.getLastStepVehicleNumber("S_0")
# east_lane = traci.lane.getLastStepVehicleNumber("E_0")
# west_lane = traci.lane.getLastStepVehicleNumber("W_0")
trafic_light_ids = traci.trafficlight.getIDList()

state = np.array(get_state(lane_ids))
action=-1
# print(state)
wait_times.append(0)
for step in range(100000): ## TO CHANGED
    #epsilon = max(1 - episode / 10, 0.01)
    nom_du_feu= traci.trafficlight.getIDList()[0]

    if step%2000 == 0:
        state=np.array(get_state(lane_ids))
        #action = epsilon_greedy_policy(state,0)*2

        action = epsilon_greedy_policy(state, 0)
        feu_1 = action // 4

        feu_2 = action % 4

        print(feu_1, feu_2)
        traci.trafficlight.setPhase(trafic_light_ids[0],2*feu_1)
        traci.trafficlight.setPhase(trafic_light_ids[1],2*feu_2)

    traci.simulationStep()

traci.close()


 Retrying in 1 seconds
[-1174317.5 -1389504.9 -1190995.1 -1264893.6 -1502944.1 -1117035.9
 -1158388.6 -1411768.2 -1249817.5 -1184549.5 -1455171.  -1516083.5
 -1233465.  -1042434.  -1548828.5 -1298637.4]
3 1
[ 310.51138    56.05644   165.10254   409.3902   -525.9904     67.984985
 -345.41342    27.622292  131.98404   142.5986    -37.121964  244.83592
 -371.61554   400.06616    -7.833185   79.89509 ]
0 3
[ 435.3722    292.66153   364.41583   229.09305  -160.83034   424.9929
   51.06063   206.40483   286.81775   304.92346    91.207535  194.54468
  344.8679    241.63489   290.19812    89.30512 ]
0 0
[ 510.81528   354.65396   357.05847   473.34875  -477.52066   301.64813
 -138.30446   122.154564  294.172     284.77472    97.63226   299.25586
  -42.9431    526.9489    137.54391    75.889496]
3 1
[1514.9158  1555.8221  1283.1481  1324.3247  -125.71943 1349.2446
  717.2221   886.67883 1205.0552  1183.7981   975.4447  1187.0645
 1134.9073  1327.0413  1091.2661   538.0683 ]
0 1
[1877.9149  1713.